In [1]:
from jupyter_dash import JupyterDash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import dash_leaflet as dl
import plotly.express as px
import pandas as pd
from crud_module import AnimalShelter
import base64
import os

# -----------------------
# MongoDB connection setup
# -----------------------
username = "aacuser"
password = "SNHU1234"
host = "nv-desktop-services.apporto.com"
port = 32648
database = "AAC"
collection = "animals"

shelter = AnimalShelter(username, password, host, port, database, collection)

# -----------------------
# Rescue Type Criteria
# -----------------------
rescue_criteria = {
    'water': ["Labrador Retriever", "Chesapeake Bay Retriever", "Newfoundland"],
    'wilderness': ["German Shepherd", "Alaskan Malamute"],
    'disaster': ["Doberman Pinscher", "German Shepherd", "Rottweiler"],
    'tracking': ["Bloodhound", "Coonhound", "Beagle"]
}

# -----------------------
# Logo Image Encoding
# -----------------------
image_path = "Grazioso Salvare Logo.png"  #Logo
if os.path.exists(image_path):
    encoded_image = base64.b64encode(open(image_path, 'rb').read()).decode('ascii')
else:
    encoded_image = None

# Initial dataset
df = pd.DataFrame.from_records(shelter.read({}))
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

# -----------------------
# App Layout
# -----------------------
app = JupyterDash('Grazioso Dashboard')

app.layout = html.Div([
    html.Div([
        html.Img(
            src='data:image/png;base64,{}'.format(encoded_image),
            style={'height': '100px', 'margin': 'auto', 'display': 'block'}
        ) if encoded_image else html.Div("Logo image not found."),
        html.Center(html.P("Dashboard created by Damon Harris", style={'fontSize': '18px', 'fontStyle': 'italic'})),
        html.Center(html.H1("Grazioso Salvare Dashboard")),
        html.Hr(),
    ]),

    html.Label("Rescue Type Filter"),
    dcc.Dropdown(
        id='filter-type',
        value='all',  # default view
        options=[
            {'label': 'Show All', 'value': 'all'},
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Wilderness Rescue', 'value': 'wilderness'},
            {'label': 'Disaster Rescue', 'value': 'disaster'},
            {'label': 'Tracking', 'value': 'tracking'}
        ],
        placeholder="Choose a rescue type"
    ),

    html.Br(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        sort_action='native',
        row_selectable='single',
        selected_rows=[0],
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left', 'minWidth': '100px'},
        style_data_conditional=[]
    ),

    html.Hr(),
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

# -----------------------
# Callbacks
# -----------------------

@app.callback(
    Output('datatable-id', 'data'),
    Input('filter-type', 'value')
)
def update_table(rescue_type):
    if rescue_type and rescue_type != 'all':
        breeds = rescue_criteria[rescue_type]
        query = {
            "animal_type": "Dog",
            "breed": {"$in": breeds},
            "age_upon_outcome_in_weeks": {"$lte": 104}
        }
        results = shelter.read(query)
    else:
        results = shelter.read({})

    df = pd.DataFrame.from_records(results)
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)
    return df.to_dict('records')


@app.callback(
    Output('graph-id', "children"),
    Input('datatable-id', "derived_virtual_data")
)
def update_graph(viewData):
    if not viewData:
        return html.P("No data to display.")
    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(dff, names='breed', title='Preferred Dog Breeds Distribution')
    return dcc.Graph(figure=fig)


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if not viewData or not index:
        return [html.P("No location selected.")]
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0]

    try:
        lat = dff.iloc[row]['location_lat']
        lon = dff.iloc[row]['location_long']
        breed = dff.iloc[row]['breed']
        name = dff.iloc[row]['name']
    except Exception:
        return [html.P("Incomplete location data.")]

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(breed),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(name)
                ])
            ])
        ])
    ]

# -----------------------
# Run Server
# -----------------------
if __name__ == '__main__':
    app.run_server(debug=True)


Connected to MongoDB.
Dash app running on http://127.0.0.1:27574/
